In [2]:
import numpy

class matrix_factorization():
    
    def __init__(self,R,a,b,it,k):
        self.R=R
        self.user_n, self.item_n=R.shape
        self.alpha=a
        self.beta=b
        self.iterations=it
        self.K=k
        
    def learn(self):
        self.b_users=numpy.zeros(self.user_n)
        self.b_items=numpy.zeros(self.item_n)
        self.b=numpy.mean(self.R[numpy.where(self.R!=0)])
        
        self.P=numpy.random.normal(scale=1./self.K,size=(self.user_n,self.K))
        self.Q=numpy.random.normal(scale=1./self.K,size=(self.item_n,self.K))
        
        self.samples=[]
        for i in range (self.user_n):
            for j in range (self.item_n):
                if (self.R[i][j])>0:
                    self.samples.append((i,j,self.R[i][j]))

        for i in range (self.iterations):
            numpy.random.shuffle(self.samples)
            self.sgd()
            
            
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            self.b_users[i]+=self.alpha*(e-self.beta*self.b_users[i])
            self.b_items[j]+=self.alpha*(e-self.beta*self.b_items[j])
            
            self.P[i,:]+=self.alpha*(e*self.Q[j,:]-self.beta*self.P[i,:])
            self.Q[j,:]+=self.alpha*(e*self.P[i,:]-self.beta*self.Q[j,:])
            
    def get_rating(self,i,j):
        return self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
    
    def complete(self):
        return self.P.dot(self.Q.T)+self.b+self.b_users[:,numpy.newaxis]+self.b_items[numpy.newaxis,:]
        

In [3]:
M = numpy.array([
    [5, 2, 0, 1],
    [1, 1, 5, 1],
    [1, 1, 0, 1],
    [1, 0, 0, 4],
    [5, 1, 0, 2],
])

mf = matrix_factorization(M, 0.1, 0.01, 20, 2)

mf.learn()

print(mf.complete())


[[5.0232846  1.7696894  5.61837472 1.11331502]
 [0.9932678  1.22900303 4.92770005 0.84584631]
 [1.05283797 0.72470391 3.26698243 1.15163339]
 [1.00870971 2.30251497 3.24397104 3.96423979]
 [4.95680353 1.27910346 3.42327927 1.83597128]]
